## Запуск моделей на своём GPU (в том числе и в первую очередь в облаке)

Вчера мы использовали облачные LLM-модели. При этом выбор моделей был ограничен, и мы не могли использовать какие-то экспериментальные модели, до-обученные энтузиастами, или нами. В облаке также есть вычислительные ресурсы - машины с GPU - которые позволяют запускать любые не слишком тяжеловесные модели.

Большой каталог моделей на разные случаи жизни есть на портале [HuggingFace](http://huggingface.com). Их работа обеспечивается библиотеками `transformers` (для текстовых моделей) и `diffusers` (для картинок).

**Для запуска в Datasphere важно выполнить следующую ячейку**. Для работы в Google Colab этого лучше не делать.

In [ ]:
%pip install --upgrade torch torchvision --index-url https://download.pytorch.org/whl/cu118
%pip install transformers accelerate

Следующую ячейку необходимо выполнить и в Google Colab:

In [ ]:
%pip install bitsandbytes

С помощью библиотеки transformers мы можем легко загружать модели с портала HuggingFace - достаточно взять оттуда фрагмент кода и его использовать (возможно, с минимальными изменениями). При этом сами модели автоматически будут скачаны и использованы.

Возьмём для примера [модель Llama 3, до-обученную на русском датасете Saiga](https://huggingface.co/IlyaGusev/saiga_llama3_8b).

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

model_path = "IlyaGusev/saiga_llama3_8b"

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=BitsAndBytesConfig(load_in_4bit=True),
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_path)

messages = [
    {"role": "system", "content": "Ты помощник, задача которого - вежливо отвечать на вопросы. Используй бодрый тон."},
    {"role": "user", "content": "Расскажи анекдот про Python"}
]

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

output = pipe(messages, **generation_args)
print(output[0]['generated_text'])


Поэкспериментируем с разной температурой генерации:

In [7]:
from transformers import GenerationConfig

generation_config = GenerationConfig.from_pretrained(model_path)
generation_config.temperature = 10.0
print(generation_config)


GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": 128009,
  "max_new_tokens": 1536,
  "pad_token_id": 128000,
  "repetition_penalty": 1.12,
  "temperature": 10.0,
  "top_k": 30,
  "top_p": 0.9
}



In [8]:
inputs = ["Почему трава зеленая?", "Сочини длинный рассказ, обязательно упоминая следующие объекты. Дано: Таня, мяч"]
for query in inputs:
    prompt = tokenizer.apply_chat_template([{
        "role": "system",
        "content": "Ты - ассистент, который всё знает"
    }, {
        "role": "user",
        "content": query
    }], tokenize=False, add_generation_prompt=True)
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(**data, generation_config=generation_config)[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True).strip()
    print(query)
    print(output)
    print()
    print("==============================")
    print()

Почему трава зеленая?
Основанный исключив его элемент, зелонный поков наше отрашение — потому что оно состоит циновкой зелнотки.

Который имеет специаэнтное поговорите "магнитоппропо" свой и, следсую жею как таксация лидий по фиейм.

Бывает пеерви сдвиги.
Чееное клювик, что-то разли.
Зеленые фотодо. Даровой и федевал
У них много нестоных тесной хозняк 
На ЗемлЕ это то и не другия, все равно трепадает он же обожание мур.
Смешало в одном из атомика (кажде для вас зельная! Но это есть 
Реча к слов) (и не более дабит). Иначе о байя в чрезводиные (для каждое  и не).
Зее. Ниже: 
Мол, они вышло за слаь 
Не будет он счастились енти.
В этом виде сего 
Форта для этой вещдеть
Вышла златник
Несущественный смотре
То уже было без него 

Это только часть ответственного 
И нгранная.
Это был. 

Принес мне все больше зелени для многих лет в себе я, я говорлю.

Этопов и не видемых и тем более их наруалины на свое время на всех земном мире и каждый момент в том насыка.

Когдати вам живы
Я бы хотел объясне

Чтобы упростить диалог с этой моделью и сохранять историю переписки, опишем небольшой класс `Agent`:

In [10]:
class Agent:
  def __init__(self,instruction):
    self.messages = [
       {"role": "system", "content": instruction}
    ]
    self.generation_args = {
      "max_new_tokens": 500,
      "return_full_text": False,
      "temperature": 0.3,
      "do_sample": True,
    }

  def __call__(self,x):
    self.messages.append({ "role" : "user", "content" : x })
    res = pipe(self.messages,**self.generation_args)
    res = res[0]['generated_text']
    self.messages.append({ "role" : "assistant", "content" : res })
    return res

teacher = Agent("Ты учитель математики по имени Мисс Радиус. Отвечай на вопросы ученика")

teacher("Здравствуйте! Я хочу узнать, что такое число Пи.")

'Здравствуйте! Число Пи — это математическая константа, которая представляет собой отношение длины окружности к диаметру круга. Это число является неотъемлемой частью многих областей математики и физики, включая геометрию, тригонометрию, аналитику и другие.\n\nЧисло Пи приблизительно равно 3.14159 (или 22/7 для более простых расчетов), но оно является бесконечно длинной десятичной дробью, так как является иррациональным числом. Это означает, что его десятичные цифры не повторяются в бесконечном порядке, делая его сложно точно вычислить.\n\nПи играет ключевую роль во многих формулах и теориях, особенно в математике и инженерии. Например, формула Аполлония для площади и периметра круга использует число Пи: \\(A = r^2 \\pi\\) и \\(P = 2r\\pi\\).\n\nНадеюсь, эта информация помогла вам лучше понять значение числа Пи. Если у вас есть еще вопросы или нужна дополнительная помощь, пожалуйста, задавайте их!'

In [ ]:
teacher("А если округлить его до целого?")

Реализуем диалог двух агентов между собой - учителя и ученика:

In [11]:
student = Agent("Ты ученик 7-го класса, который хочет узнать больше про математику. Ты разговариваешь с учителем.  Используй короткие разговорные фразы.")
teacher = Agent("Ты учитель математики по имени Мисс Радиус. Отвечай на вопросы ученика 7-го класса. Используй короткие разговорные фразы.")

msg = "Здравствуйте! Я хочу узнать, что такое число Пи."
for _ in range(5):
  print(f"Ученик: {msg}")
  msg = teacher(msg)
  print(f"Учитель: {msg}")
  msg = student(msg)

Ученик: Здравствуйте! Я хочу узнать, что такое число Пи.
Учитель: Привет! Число Пи — это математическая константа, которая равна примерно 3.14159 (но она бесконечно длинная!). Это число связано с площадью и периметром кругов. Например, если у тебя есть круг радиуса R, то его длина окружности будет примерно 2 * π * R. Надеюсь, теперь ты лучше понимаешь, как работает Пи!
Ученик: Ой, спасибочки за объяснение! Я понял, что Пи важный для геометрии и помогает найти длину окружностей кругов. Если я хочу найти длину окружности круга с радиусом 5 см, нужно ли мне умножить на 10?
Учитель: Неправильно! Чтобы найти длину окружности круга с радиусом 5 см, нужно использовать формулу: длина = 2 * π * R. В данном случае, R равно 5 см, так что длина будет 2 * 3.14159 * 5 = 31.4159 см. Не забывай, что это число Пи в точности не заканчивается на 3.14, но мы используем эту приближенную версию.
Ученик: Спасибочки за исправление! Я вижу, что я ошибся в расчете. Действительно, длина окружности круга с радиус

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Учитель: Пожалуйста! Всегда рад помочь. Если у тебя возникнут еще вопросы, не стесняйся обращаться. Удачного дня!
Ученик: Спасибо большое! Удачного дня тебе тоже!
Учитель: С удовольствием! Удачного дня тебе тоже!


Все сообщения доступны в поле `messages`:

In [12]:
teacher.messages

[{'role': 'system',
  'content': 'Ты учитель математики по имени Мисс Радиус. Отвечай на вопросы ученика 7-го класса. Используй короткие разговорные фразы.'},
 {'role': 'user',
  'content': 'Здравствуйте! Я хочу узнать, что такое число Пи.'},
 {'role': 'assistant',
  'content': 'Привет! Число Пи — это математическая константа, которая равна примерно 3.14159 (но она бесконечно длинная!). Это число связано с площадью и периметром кругов. Например, если у тебя есть круг радиуса R, то его длина окружности будет примерно 2 * π * R. Надеюсь, теперь ты лучше понимаешь, как работает Пи!'},
 {'role': 'user',
  'content': 'Ой, спасибочки за объяснение! Я понял, что Пи важный для геометрии и помогает найти длину окружностей кругов. Если я хочу найти длину окружности круга с радиусом 5 см, нужно ли мне умножить на 10?'},
 {'role': 'assistant',
  'content': 'Неправильно! Чтобы найти длину окружности круга с радиусом 5 см, нужно использовать формулу: длина = 2 * π * R. В данном случае, R равно 5 см,

## Модели синтеза речи

На портале HuggingFace можно найти разные интересные модели, например, модель для синтеза речи от Suno, создателей инструмента генерации музыки. Попробуем синтезировать простую фразу:

In [13]:
from transformers import pipeline
import scipy

synthesiser = pipeline("text-to-speech", "suno/bark-small")

speech = synthesiser("Привет! Моя собака круче тебя!", forward_params={"do_sample": True})


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/353 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cuda:0
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Чтобы послушать результат, нужно использовать небольшую Python-магию:

In [16]:
from IPython.display import Audio

speech = synthesiser("Привет! Моя собака круче тебя! [Bark]", forward_params={"do_sample": True})
Audio(speech['audio'], rate=speech['sampling_rate'])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


Можно попробовать и загрузить большую модель синтеза - у неё качество повыше, но сильно дольше время синтеза. Также обернём её в функцию и сделаем так, чтобы она сразу возвращала AudioSegment, который потом можно монтировать. Большая модель позволяет также передать [название голосового пресета](https://suno-ai.notion.site/8b8e8749ed514b0cbf3f699013548683?v=bc67cff786b04b50b3ceb756fd05f68c), который нужно использовать. Образцы кода адаптированы со [странички модели suno/bark](https://huggingface.co/suno/bark).

In [21]:
from transformers import AutoProcessor, BarkModel
from pydub import AudioSegment
import torch
import io

# Sampling Rate, используемый в bark
SAMPLING_RATE=22000

# Смотрим, доступен ли GPU
device = "cuda" if torch.cuda.is_available() else "cpu"

# Создаём модели
processor = AutoProcessor.from_pretrained("suno/bark")
model = BarkModel.from_pretrained("suno/bark")

# Переносим вычисления на GPU, если есть
_ = model.to(device)

In [30]:
def synthesize(x,voice_preset='v2/en_speaker_1'):
  inputs = processor(x, voice_preset=voice_preset).to(device)
  audio_array = model.generate(**inputs,do_sample=True)
  speech = audio_array.cpu().numpy().squeeze()
  return Audio(speech,rate=SAMPLING_RATE) #AudioSegment(speech.tobytes(), sample_width=speech.dtype.itemsize, channels=1, frame_rate=SAMPLING_RATE)

synthesize("Hello, my dog is cute! [laughs] I actually love it more than my parents!")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


In [31]:
synthesize("О, моя собака так крута! Я люблю её, и ещё конечно мою бабушку!",voice_preset='v2/ru_speaker_9')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


Другой способ синтеза речи - это использовать какую-нибудь библиотеку, например, Silero Voice:

In [32]:
%pip install -q silero-tts

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.6/91.6 kB 6.3 MB/s eta 0:00:00


Посмотрим, какие модели доступны:

In [33]:
from silero_tts.silero_tts import SileroTTS

models = SileroTTS.get_available_models()
print("Available models:", models)

2025-10-01 17:43:46.505 | WARNING  | silero_tts.silero_tts:get_available_models:309 - Models config file not found: /usr/local/lib/python3.12/dist-packages/silero_tts/latest_silero_models.yml. Downloading...
2025-10-01 17:43:46.688 | SUCCESS  | silero_tts.silero_tts:download_models_config_static:361 - Models config file downloaded: /usr/local/lib/python3.12/dist-packages/silero_tts/latest_silero_models.yml


Available models: {'ru': ['v4_ru', 'v3_1_ru', 'ru_v3', 'aidar_v2', 'aidar_8khz', 'aidar_16khz', 'baya_v2', 'baya_8khz', 'baya_16khz', 'irina_v2', 'irina_8khz', 'irina_16khz', 'kseniya_v2', 'kseniya_8khz', 'kseniya_16khz', 'natasha_v2', 'natasha_8khz', 'natasha_16khz', 'ruslan_v2', 'ruslan_8khz', 'ruslan_16khz'], 'en': ['v3_en', 'v3_en_indic', 'lj_v2', 'lj_8khz', 'lj_16khz'], 'de': ['v3_de', 'thorsten_v2', 'thorsten_8khz', 'thorsten_16khz'], 'es': ['v3_es', 'tux_v2', 'tux_8khz', 'tux_16khz'], 'fr': ['v3_fr', 'gilles_v2', 'gilles_8khz', 'gilles_16khz'], 'ba': ['aigul_v2'], 'xal': ['v3_xal', 'erdni_v2'], 'tt': ['v3_tt', 'dilyara_v2'], 'uz': ['v4_uz', 'v3_uz', 'dilnavoz_v2'], 'ua': ['v4_ua', 'v3_ua', 'mykyta_v2'], 'indic': ['v4_indic', 'v3_indic'], 'cyrillic': ['v4_cyrillic'], 'multi': ['multi_v2']}


И какие спикеры:

In [34]:
tts = SileroTTS(model_id='v4_ru', language='ru', sample_rate=48000, device='cuda')
tts.get_available_speakers()


2025-10-01 17:43:49.291 | SUCCESS  | silero_tts.silero_tts:load_models_config:48 - Models config loaded from: /usr/local/lib/python3.12/dist-packages/silero_tts/latest_silero_models.yml
2025-10-01 17:43:49.298 | INFO     | silero_tts.silero_tts:init_model:148 - Initializing model
2025-10-01 17:43:49.301 | INFO     | silero_tts.silero_tts:init_model:156 - Using 1 GPU(s)...
2025-10-01 17:43:49.351 | INFO     | silero_tts.silero_tts:init_model:175 - Downloading model from https://models.silero.ai/models/tts/ru/v4_ru.pt to /usr/local/lib/python3.12/dist-packages/silero_tts/silero_models/v4_ru_ru.pt
2025-10-01 17:43:50.416 | SUCCESS  | silero_tts.silero_tts:init_model:181 - Model downloaded successfully.
2025-10-01 17:43:50.417 | INFO     | silero_tts.silero_tts:init_model:187 - Loading model
2025-10-01 17:43:54.536 | INFO     | silero_tts.silero_tts:init_model:192 - Model to device takes 4.11 seconds
2025-10-01 17:43:54.539 | INFO     | silero_tts.silero_tts:init_model:195 - Synchronizing 

['aidar', 'baya', 'kseniya', 'xenia', 'eugene', 'random']

Сделаем удобную функцию для синтеза, которая будет возвращать аудиосегменты, которые можно будет склеивать вместе:

In [35]:
import io
from pydub import AudioSegment

def synth(x,speaker=None):
  if speaker:
    tts.change_speaker(speaker)
  bio = io.BytesIO()
  tts.tts(x,bio)
  bio.seek(0)
  return AudioSegment(bio.getvalue())

synth('Привет')

2025-10-01 17:43:55.730 | INFO     | silero_tts.silero_tts:preprocess_text:234 - Preprocessing text
2025-10-01 17:43:55.737 | INFO     | silero_tts.silero_tts:init_wave_file:289 - Initializing wave file: <_io.BytesIO object at 0x7a3613527b50>
2025-10-01 17:43:55.750 | INFO     | silero_tts.silero_tts:tts:271 - Starting TTS
2025-10-01 17:43:55.762 | INFO     | silero_tts.silero_tts:tts:274 - Processing line 1/1: Привет
2025-10-01 17:44:03.885 | SUCCESS  | silero_tts.silero_tts:tts:286 - Speech saved to <_io.BytesIO object at 0x7a3613527b50>


Теперь можем озвучить диалог:

In [ ]:
res = synth('Вот какой диалог получился однажды у учителя и ученика!','baya')
for x in teacher.messages[1:]:
  spk = 'xenia' if x['role']=='assistant' else 'aidar'
  res += synth(x['content'],spk)

res

## Мораль

Развёртывание своих моделей требует существенных усилий, и результат не всегда сразу хорош. Облако в этом смысле сильно упрощает жизнь, предоставляя готовые модели. Но развёртывая свои модели мы можем экспериментировать, важно лишь помнить об ограниченных вычислительных мощностях.